<a href="https://colab.research.google.com/github/yogasgm/user-review-analysis-framework/blob/main/Generate_App_Improvement_Recommendation_with_BERTopic_and_GPT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting to Google Drive for data storage and retrieval
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Reading CSV file
file_path = '/content/drive/My Drive/Colab Notebooks/dataset/linkaja_reviews.csv'
df_raw = pd.read_csv(file_path)

In [ ]:
# Filter data with appVersion 4.27.0
df = df_raw.loc[df_raw['appVersion'] == '4.27.0']

# Check if there is data with appVersion 4.27.0
if not df.empty:
    # Take the earliest and latest dates
    earliest_date = df['at'].min()
    latest_date = df['at'].max()

    print(f"Periode penggunaan versi 4.27.0:")
    print(f"Tanggal paling awal: {earliest_date}")
    print(f"Tanggal paling akhir: {latest_date}")
else:
    print("Tidak ada data dengan appVersion 4.27.0")

Periode penggunaan versi 4.27.0:
Tanggal paling awal: 2022-09-10 06:45:01
Tanggal paling akhir: 2024-03-12 06:59:35


In [ ]:
reviews = df["content"].str.lower()
reviews = reviews[reviews.str.split().str.len() >= 3]
reviews = reviews.reset_index(drop=True)
print(f"Number of reviews after preprocessing: {len(reviews)}")

Number of reviews after preprocessing: 5627


In [ ]:
# Installing Required Packages
!pip install bertopic openai adjustText --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 14.0 MB/s eta 0:00:00


In [ ]:
import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

In [ ]:
from google.colab import userdata
client = openai.OpenAI(api_key=userdata.get('openai_key'))

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
reviews = reviews.reset_index(drop=True)

In [ ]:
embeddings = embedding_model.encode(reviews, show_progress_bar=True)

Batches:   0%|          | 0/176 [00:00<?, ?it/s]

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# Download stopwords if not yet available
nltk.download('stopwords')
indo_stopwords = set(stopwords.words('indonesian'))

# Define CountVectorizer with Indonesian stopwords from NLTK
vectorizer_model = CountVectorizer(stop_words=list(indo_stopwords))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the following format:
topic: <topic label>
"""

In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

# KeyBERT
keybert = KeyBERTInspired()

# MMR
mmr = MaximalMarginalRelevance(diversity=0.3)

# Text generation with GPT
gpt = OpenAI(client, model="gpt-4o", chat=True, prompt=prompt, delay_in_seconds=10)

# All representation models
representation_model = {
    "KeyBERT": keybert,
    "GPT 4": gpt,
    "MMR": mmr,
}

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model, #stopwords removal
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(reviews, embeddings)

2024-07-27 12:37:48,967 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-27 12:38:01,360 - BERTopic - Dimensionality - Completed ✓
2024-07-27 12:38:01,363 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-27 12:38:01,645 - BERTopic - Cluster - Completed ✓
2024-07-27 12:38:01,674 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 115/115 [20:29<00:00, 10.70s/it]
2024-07-27 13:13:49,735 - BERTopic - Representation - Completed ✓


In [ ]:
# Show topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,GPT 4,MMR,Representative_Docs
0,-1,1838,-1_aja_link_banget_top,"[aja, link, banget, top, linkaja, aplikasi, up...","[aplikasinya, aplikasi, membantu, tagihan, bag...",[LinkAja Application Feedback],"[link, top, linkaja, aplikasi, pembayaran, upg...",[paling suka sama fitur pembayaran via qris. a...
1,0,192,0_syariah_berkah_alhamdulillah_layanan,"[syariah, berkah, alhamdulillah, layanan, aman...","[syariahnya, alhamdulillah, alhamdulilah, laya...",[Positive Feedback on LinkAja Syariah Services],"[syariah, berkah, alhamdulillah, layanan, nabu...",[untuk layanan syariah dari linkaja semua lanc...
2,1,135,1_ojk_tenang_awasi_diawasi,"[ojk, tenang, awasi, diawasi, hati, mendanai, ...","[tenang, ojk, aplikasinya, ok, bermanfaat, kha...",[keamanan aplikasi pendanaan diawasi oleh OJK],"[diawasi, mendanai, khawatir, aplikasinya, don...","[aplikasi ini diawasi ojk , jadi saya bisa men..."
3,2,102,2_bertransaksi_mudah_membantu_mempermudah,"[bertransaksi, mudah, membantu, mempermudah, m...","[bertransaksi, transaksinya, membantu, bermanf...",[Kemudahan dalam Bertransaksi],"[bertransaksi, membantu, mempermudah, perbanka...","[sangat membantu dalam bertransaksi, sangat me..."
4,3,100,3_saldo_masuk_link_isi,"[saldo, masuk, link, isi, rekening, bank, berh...","[saldonya, masuk, rekening, saldo, penjelasann...",[saldo tidak masuk setelah top up atau transfer],"[saldo, masuk, rekening, sukses, transfer, bri...","[rekomend banget buat link aja , kalo bisa , t..."
...,...,...,...,...,...,...,...,...
110,109,11,109_okeii_kecanggihan_biarpun_perkenbanganya,"[okeii, kecanggihan, biarpun, perkenbanganya, ...","[kecanggihan, tingkatkan, performanya, fiturny...",[LinkAja Innovation and Performance],"[okeii, kecanggihan, perkenbanganya, fiturnya,...",[harus tetap berinovasi.. biarpun sekarang sud...
111,110,11,110_bayak_ekonomis_bisaa_mudah,"[bayak, ekonomis, bisaa, mudah, dipermudah, li...","[dipermudah, murah, linkaja, akses, mudah, kem...",[kemudahan dan kepraktisan penggunaan LinkAja],"[ekonomis, dipermudah, pembayaran, kemudahan, ...",[pembayaran menjadi lebih mudah dengan link aj...
112,111,11,111_kebutuhan_job_fungsi_bener,"[kebutuhan, job, fungsi, bener, memenuhi, good...","[aplikasi, makasih, bermanfaat, membantu, appl...",[Keunggulan dan Kepuasan Pengguna LinkAja seba...,"[kebutuhan, job, arigatou, berjuta, lanjutkan,...",[app ini bisa jadi solusi buat macem2 kebutuha...
113,112,11,112_kendala_penanganan_konsultasi_bentul2,"[kendala, penanganan, konsultasi, bentul2, dib...","[aplikasinya, aplikasi, bermasalah, aksesnya, ...",[User Experience and Transaction Efficiency in...,"[kendala, penanganan, konsultasi, bentul2, dib...",[transaksi jadi lebih mudah akan tetapi dibebe...


In [ ]:
# Ekspor ke CSV dalam lingkungan Colab
topic_info = topic_model.get_topic_info()
output_path = 'topic_info.csv'
topic_info.to_csv(output_path, index=False)

In [ ]:
gpt_labels = [label[0][0].split("\n")[0] for label in topic_model.get_topics(full=True)["GPT 4"].values()]
topic_model.set_topic_labels(gpt_labels)

In [ ]:
topic_model.visualize_documents(reviews, reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=False, custom_labels=True)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
gpt_labels

['LinkAja Application Feedback',
 'Positive Feedback on LinkAja Syariah Services',
 'keamanan aplikasi pendanaan diawasi oleh OJK',
 'Kemudahan dalam Bertransaksi',
 'saldo tidak masuk setelah top up atau transfer',
 'Transaction Failures and Balance Issues',
 'Delayed Transaction Processing and Poor Customer Service',
 'aplikasi tidak bisa dibuka',
 'upgrade full service duration',
 'PIN verification and account blocking issues',
 'login and access issues with an account',
 'LinkAja for Daily Financial Transactions',
 'aplikasi transaksi keuangan online',
 'Positive Reviews of the LinkAja App',
 'Dompet Digital LinkAja - Praktis dan Aman',
 'positive feedback on apk quality',
 'kemudahan dan kecepatan',
 'upgrade process delay',
 'delayed full service upgrade',
 'positive user experience and minimal errors',
 'positive feedback',
 'Difficulty in Full Service Account Upgrade Verification Process',
 'Kemudahan Pembayaran dan Pembelian Pulsa dengan LinkAja',
 'Kemudahan Pembayaran dengan

In [ ]:
representative_docs = []
for topic_num in topic_model.get_topics(full=True)["GPT 4"].keys():
    # Get the first representative document for the topic
    first_representative_doc = topic_model.get_representative_docs(topic_num)[0]
    representative_docs.append(first_representative_doc)


In [ ]:
def get_gpt4_response(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt,
        temperature=0.0,  # Ensures deterministic output
        max_tokens=50  # Sufficient for concise classification responses
    )
    # Assuming the last message in the response is the assistant's response
    # and accessing its content attribute directly
    last_message_content = response.choices[0].message.content.strip()
    return last_message_content

In [ ]:
classification_results = []

for topic_num, topic_name, representative_doc in zip(topic_model.get_topics(full=True)["GPT 4"].keys(), gpt_labels, representative_docs):
    chat_prompt = [
        {"role": "system", "content": "Your task is to analyze the given topic and its representative review from a mobile application and provide classifications based on the following criteria:\n\n1. Sentiment Analysis:\n- Classify the sentiment expressed in the representative review as 'positive', 'negative', or 'neutral' towards the mobile application.\n- Consider the overall tone, emotions, and satisfaction level conveyed in the review.\n\n2. Topic Relevance Assessment:\n- Determine if the topic is 'relevant' or 'not relevant' to specific technical issues, bugs, performance problems, usability concerns, or other concrete aspects that can be addressed by the developers to improve the mobile application.\n- Topics that are too general, vague, or not directly related to actionable issues should be classified as 'not relevant'.\n\nProvide your analysis in the following format:\n<sentiment>, <relevance>"},
        {"role": "user", "content": "- Topic: App crashes frequently.\n- Review: This app is always crashing and it's really frustrating."},
        {"role": "assistant", "content": "negative, relevant"},
        {"role": "user", "content": "- Topic: Slow transaction processing.\n- Review: The transaction processing in this app is often slow and takes a long time to complete."},
        {"role": "assistant", "content": "negative, relevant"},
        {"role": "user", "content": "- Topic: Negative app reviews.\n- Review: I don't like this app. It's terrible."},
        {"role": "assistant", "content": "negative, not relevant"},
        {"role": "user", "content": f"- Topic: {topic_name}\n- Review: {representative_doc}."}
    ]

    response = get_gpt4_response(chat_prompt)
    classification_results.append((topic_num, topic_name, representative_doc, response))

# Convert Classification Result to DataFrame
df_classification = pd.DataFrame(classification_results, columns=["Topic Number", "Topic Name", "Representative Doc", "Classification"])

In [ ]:
# Split the 'Classification' column into two new columns 'Class' and 'Sentiment'
df_classification[['Sentiment', 'Relevansi']] = df_classification['Classification'].str.split(', ', expand=True)

df_classification

,Topic Number,Topic Name,Representative Doc,Classification,Sentiment,Relevansi
0,-1,LinkAja Application Feedback,paling suka sama fitur pembayaran via qris. am...,"positive, not relevant",positive,not relevant
1,0,Positive Feedback on LinkAja Syariah Services,untuk layanan syariah dari linkaja semua lanca...,"positive, not relevant",positive,not relevant
2,1,keamanan aplikasi pendanaan diawasi oleh OJK,"aplikasi ini diawasi ojk , jadi saya bisa mend...","positive, not relevant",positive,not relevant
3,2,Kemudahan dalam Bertransaksi,sangat membantu dalam bertransaksi,"positive, not relevant",positive,not relevant
4,3,saldo tidak masuk setelah top up atau transfer,"rekomend banget buat link aja , kalo bisa , to...","positive, relevant",positive,relevant
...,...,...,...,...,...,...
110,109,LinkAja Innovation and Performance,harus tetap berinovasi.. biarpun sekarang suda...,"positive, not relevant",positive,not relevant
111,110,kemudahan dan kepraktisan penggunaan LinkAja,pembayaran menjadi lebih mudah dengan link aja,"positive, not relevant",positive,not relevant
112,111,Keunggulan dan Kepuasan Pengguna LinkAja sebag...,app ini bisa jadi solusi buat macem2 kebutuhan...,"positive, not relevant",positive,not relevant
113,112,User Experience and Transaction Efficiency in ...,transaksi jadi lebih mudah akan tetapi dibeber...,"neutral, relevant",neutral,relevant


In [ ]:
# Ekspor df_classification ke CSV dalam lingkungan Colab
output_path = 'df_classification.csv'
df_classification.to_csv(output_path, index=False)

In [ ]:
# Filter to get topics that are 'relevant' and with 'negative' Sentiment
df_relevant_topics = df_classification[(df_classification['Sentiment'] == 'negative') & (df_classification['Relevansi'] == 'relevant')].copy()

# Ask if there are any topics that need revision
user_response = input("Apakah ada topik yang ingin Anda revisi? (y/n): ").strip().lower()

if user_response == "y":
    # Ask for the topic number you want to revise
    topik_yang_direvisi = input("Masukkan nomor topik yang ingin Anda revisi (pisahkan dengan koma jika lebih dari satu): ")
    topik_yang_direvisi = [int(x.strip()) for x in topik_yang_direvisi.split(',') if x]

    # The revision process
    for topik in topik_yang_direvisi:
        # Check for topic existence
        if topik in df_classification['Topic Number'].values:
            # Sentiment Revision
            sentiment_baru = input(f"Masukkan sentiment baru untuk topik {topik} (positive/negative/neutral): ").strip().lower()
            df_classification.loc[df_classification['Topic Number'] == topik, 'Sentiment'] = sentiment_baru

            # Class Revision
            klasifikasi_baru = input(f"Masukkan klasifikasi baru untuk topik {topik} (relevant/not relevant): ").strip().lower()
            df_classification.loc[df_classification['Topic Number'] == topik, 'Relevansi'] = klasifikasi_baru
        else:
            print(f"Topik {topik} tidak ditemukan.")

    # Update df_relevant_topics after revision
    df_relevant_topics = df_classification[(df_classification['Sentiment'] == 'negative') & (df_classification['Relevansi'] == 'relevant')].copy()

print("DataFrame dengan sentimen negatif dan topik yang relevant telah disimpan sebagai df_relevant_topics.")

Apakah ada topik yang ingin Anda revisi? (y/n): n
DataFrame dengan sentimen negatif dan topik yang relevant telah disimpan sebagai df_relevant_topics.


In [ ]:
df_relevant_topics.to_csv('relevant_topics.csv', index=False)
df_relevant_topics

,Topic Number,Topic Name,Representative Doc,Classification,Sentiment,Relevansi
5,4,Transaction Failures and Balance Issues,transaksi gagal saldo berkurang...,"negative, relevant",negative,relevant
6,5,Delayed Transaction Processing and Poor Custom...,kemaren qu kirim uang ke bank sampe sekarang p...,"negative, relevant",negative,relevant
7,6,aplikasi tidak bisa dibuka,knapa gak bisa di buka pdahal pin sudah benar,"negative, relevant",negative,relevant
8,7,upgrade full service duration,upgrade full service sangat lama,"negative, relevant",negative,relevant
9,8,PIN verification and account blocking issues,masak akun saya di blokir padahal pin benar ko...,"negative, relevant",negative,relevant
10,9,login and access issues with an account,ko ngga bisa di buka si?,"negative, relevant",negative,relevant
17,16,upgrade process delay,lama sekali upgrade nya 😤😤😤😤😤😤,"negative, relevant",negative,relevant
18,17,delayed full service upgrade,kenapa upgrade full service nya lama padahal s...,"negative, relevant",negative,relevant
21,20,Difficulty in Full Service Account Upgrade Ver...,aplikasinya diperbaiki yg bagus. upgrade akun ...,"negative, relevant",negative,relevant
24,23,Customer Service Issues with LinkAja Transacti...,ini kenapa transaksi saya pending terus? hubun...,"negative, relevant",negative,relevant


In [ ]:
# Get Representative_Docs for each relevant topic
representative_docs = {}
for topic_num in df_relevant_topics['Topic Number']:
    docs = topic_model.get_representative_docs(topic_num)
    representative_docs[topic_num] = docs

In [ ]:
# Data formatting for LLM
llm_input = ""
for topic_num, docs in representative_docs.items():
    topic_name = df_relevant_topics[df_relevant_topics['Topic Number'] == topic_num]['Topic Name'].iloc[0]
    llm_input += f"Topic {topic_num}: {topic_name}\nUser Reviews:\n"
    for i, doc in enumerate(docs):
        llm_input += f"{i+1}. {doc}\n"
    llm_input += "\n"

In [ ]:
print(llm_input)

Topic 4: Transaction Failures and Balance Issues
User Reviews:
1. transaksi gagal saldo berkurang...
2. transaksi gagal tapi saldo berkurang
3. aplikasi yg banyak masalah, kirim uang gagal mulu, aplikasi gak guna

Topic 5: Delayed Transaction Processing and Poor Customer Service
User Reviews:
1. kemaren qu kirim uang ke bank sampe sekarang pending, di suruh nunggu 1x24 jam. skarang di suruh nunggu 3x24 jam.. hufffff
2. di telpon gk bisa. di email tak ada respon. chat juga gak bisa. bagaimana ini. saya transfer sudah 1x24 jam pending.
3. minus bgt. top up dari malam belum masuk, suruh nunggu 1x24 jam. sudah lewat 1x24 jam saldo juga belum masuk. suruh nunggu 3x24 jam. nggak kaya apk lain nya.. apk lain top up langsung masuk nggak pake pending. refund pun cuma nunggu 1 jam. ini udah top up gak masuk masuk, komplen suruh nunggu terus terusan 😑

Topic 6: aplikasi tidak bisa dibuka
User Reviews:
1. knapa gak bisa di buka pdahal pin sudah benar
2. ini kenapa ya, kok aplikasinya tiba" gk bisa

In [ ]:
# Step 1: Grouping topics into problem areas
messages_problem_identification = [
    {
        "role": "system",
        "content": "You are an AI assistant tasked with analyzing topics extracted from user reviews of a mobile application, identifying main problem areas, and grouping these topics into those identified problem areas based on the issues they discuss."
    },
    {
        "role": "user",
        "content": f"""The following are topics extracted from user reviews of a mobile application, along with representative reviews for each topic. These topics are related to various issues, complaints, and feedback from users about the application:

{llm_input}

Please analyze these topics and their representative reviews, then group them into main problem areas that the app developers should address. For each problem area, list only the related topic numbers.

IMPORTANT: Ensure that ALL topics are grouped into problem areas. Do not leave any topic ungrouped.

Format your response EXACTLY as follows, with no additional text before or after:

PROBLEM_AREA: [Name of the problem area]
TOPIC_NUMBERS: [Topic number],[Topic number],[Topic number]

PROBLEM_AREA: [Name of the problem area]
TOPIC_NUMBERS: [Topic number],[Topic number],[Topic number]
"""
    }
]

# Send the prompt to GPT-4
response_problem_areas = client.chat.completions.create(
    model="gpt-4o",
    messages=messages_problem_identification
)

# Extract the problem areas and related topics
problem_areas = response_problem_areas.choices[0].message.content
print(problem_areas)

In [ ]:
problem_areas = """PROBLEM_AREA: Transaction Failures and Payment Issues
TOPIC_NUMBERS: 4,38,43,50,52,75,76,79,81,98,103

PROBLEM_AREA: Delayed Transactions and Processing Issues
TOPIC_NUMBERS: 5,20,51,73,91

PROBLEM_AREA: Full Service Upgrade Issues
TOPIC_NUMBERS: 7,16,17,29,41,47,49,53,58,64,86

PROBLEM_AREA: PIN and Verification Issues
TOPIC_NUMBERS: 8,39,68,85

PROBLEM_AREA: Login and Access Issues
TOPIC_NUMBERS: 6,9,70,106

PROBLEM_AREA: Poor Customer Service
TOPIC_NUMBERS: 23,37,65,113

PROBLEM_AREA: Connectivity and Performance Issues
TOPIC_NUMBERS: 25,69,74,77,94,99

PROBLEM_AREA: Locking and Verification Complaints
TOPIC_NUMBERS: 42,72,82,80

PROBLEM_AREA: General User Experience and State-Owned Enterprise Criticisms
TOPIC_NUMBERS: 32,78,90"""

In [ ]:
# Parse GPT output to extract problem areas and associated topic numbers
def parse_gpt_output(problem_areas):
    parsed_problem_areas = {}
    current_area = ""

    for line in problem_areas.split('\n'):
        line = line.strip()
        if line.startswith("PROBLEM_AREA:"):
            current_area = line.split(":", 1)[1].strip()
        elif line.startswith("TOPIC_NUMBERS:"):
            topics = line.split(":", 1)[1].strip()
            topic_numbers = [int(num) for num in topics.split(',') if num.isdigit()]
            parsed_problem_areas[current_area] = topic_numbers

    return parsed_problem_areas

# Use the parsing function
parsed_problem_areas = parse_gpt_output(problem_areas)

# Create a DataFrame
problem_area_df = []
for area, topic_numbers in parsed_problem_areas.items():
    for topic_num in topic_numbers:
        if topic_num in df_relevant_topics['Topic Number'].values:
            topic_name = df_relevant_topics[df_relevant_topics['Topic Number'] == topic_num]['Topic Name'].iloc[0]
            problem_area_df.append({
                'Problem Area': area,
                'Topic Number': topic_num,
                'Topic Name': topic_name
            })

problem_area_df = pd.DataFrame(problem_area_df)

print("Parsed Problem Areas:")
problem_area_df

Parsed Problem Areas:


,Problem Area,Topic Number,Topic Name
0,Transaction Failures and Payment Issues,4,Transaction Failures and Balance Issues
1,Transaction Failures and Payment Issues,38,Masalah Pembelian Token Listrik di LinkAja
2,Transaction Failures and Payment Issues,43,LinkAja saldo top-up issues
3,Transaction Failures and Payment Issues,50,MyTelkomsel Payment Issues with LinkAja
4,Transaction Failures and Payment Issues,52,Frequent Transfer Failures
5,Transaction Failures and Payment Issues,75,e-toll top-up issues
6,Transaction Failures and Payment Issues,76,Issues with Top-Up Transactions and Customer S...
7,Transaction Failures and Payment Issues,79,masalah transaksi pengisian dan top up saldo t...
8,Transaction Failures and Payment Issues,81,ATM Withdrawal Issues
9,Transaction Failures and Payment Issues,98,LinkAja Transaction Issues and Refund Complaints


In [ ]:
# Identify topics not used by GPT in problem area grouping
used_topics = set()
for area, topic_numbers in parsed_problem_areas.items():
    used_topics.update(topic_numbers)

# Create a DataFrame for unused topics
unused_topics_df = df_relevant_topics[~df_relevant_topics['Topic Number'].isin(used_topics)].copy()

# Add a status column to indicate unused topics
unused_topics_df['Status'] = 'Unused by GPT'

# Sort unused topics by topic number
unused_topics_df = unused_topics_df.sort_values('Topic Number')

# Reset index
unused_topics_df = unused_topics_df.reset_index(drop=True)

print("Topik yang tidak digunakan oleh GPT:")
unused_topics_df

Topik yang tidak digunakan oleh GPT:


,Topic Number,Topic Name,Representative Doc,Classification,Sentiment,Relevansi,Status


In [ ]:
# Function to prepare input for GPT based on problem area and topics
def check_input_for_gpt(area, topic_numbers, representative_docs, df_relevant_topics):
    area_input = f"Problem Area: {area}\n\n"
    for topic_num in topic_numbers:
        topic_name = df_relevant_topics[df_relevant_topics['Topic Number'] == topic_num]['Topic Name'].iloc[0]
        area_input += f"Topic {topic_num}: {topic_name}\nUser Reviews:\n"
        if topic_num in representative_docs and representative_docs[topic_num]:
            for i, doc in enumerate(representative_docs[topic_num][:3], 1):
                area_input += f"{i}. {doc}\n"
        else:
            rep_doc = df_relevant_topics[df_relevant_topics['Topic Number'] == topic_num]['Representative Doc'].iloc[0]
            area_input += f"1. {rep_doc}\n"
        area_input += "\n"
    return area_input

# Function to generate recommendations for a single problem area
def generate_recommendation_for_area(client, area, area_input):
    messages_recommendation_generation = [
        {
            "role": "system",
            "content": "You are an AI assistant tasked with providing recommendations to app developers by analyzing topics and their representative reviews highlighting issue in the LinkAja mobile application. LinkAja is a digital wallet and financial services platform in Indonesia, offering features such as mobile payments, money transfers, bill payments, and e-commerce transactions."
        },
        {
            "role": "user",
            "content": f"""Here are the relevant topics and user reviews for the problem area '{area}':\n\n{area_input}

Based on these topics and reviews, please provide specific recommendations for the app developers to address the identified issues and improve the application.

Your response should include for each recommendation:
1. A specific recommendation focusing on how the developers can address the issues and improve the application.
2. The potential benefits of implementing this recommendation, such as improved user satisfaction, increased app usage, or enhanced app performance.
3. If applicable, suggest any best practices, tools, or technologies that can assist in implementing the recommendation effectively.

Please format your response EXACTLY as follows for each recommendation:

RECOMMENDATION: [Your specific recommendation]
BENEFITS: [Potential benefits of implementing the recommendation]
BEST_PRACTICES: [Suggestions for effective implementation, if applicable]

Provide recommendations that you deem necessary to address the issue comprehensively.
"""
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages_recommendation_generation
    )

    return response.choices[0].message.content

In [ ]:
# Main process for generating recommendations for each problem area
all_recommendations = {}

for area, topic_numbers in parsed_problem_areas.items():
    # Prepare and display input for the current problem area
    area_input = check_input_for_gpt(area, topic_numbers, representative_docs, df_relevant_topics)
    print(f"Input for area '{area}':")
    print(area_input)
    print("=" * 50)

    # Get confirmation before proceeding
    proceed = input("Does the input look correct? (y/n): ").strip().lower()
    if proceed != 'y':
        print(f"Skipping area '{area}'. Please review and adjust the input if necessary.")
        continue

    # Generate recommendation for the current problem area
    recommendation = generate_recommendation_for_area(client, area, area_input)
    all_recommendations[area] = recommendation

    # Display the generated recommendation
    print(f"Recommendation for area '{area}':")
    print(recommendation)
    print("=" * 50)

Input for area 'Transaction Failures and Payment Issues':
Problem Area: Transaction Failures and Payment Issues

Topic 4: Transaction Failures and Balance Issues
User Reviews:
1. transaksi gagal saldo berkurang...
2. transaksi gagal tapi saldo berkurang
3. aplikasi yg banyak masalah, kirim uang gagal mulu, aplikasi gak guna

Topic 38: Masalah Pembelian Token Listrik di LinkAja
User Reviews:
1. beli token listrik dri pagi jam 6 sampai sekarang gak masuk2 gmn ini???
2. sudah lama pakai link aja, baru ini ada kendala beli token listrik dan sudah sukses tapi token belum di kirim dan ket proses, sudah kirim kendala juga di email dan sudah 4 jam menunggu masih saja proses, di email juga belum ada balasan
3. masa iya isi token tapi no. token nya g' ada , . apa apa ini , . kembali kan saldo saya , . atau keluar kan no. token nya , . mana ini kembali kan link aja saya , . disini bisa kirim bukti g' , . klo no. token nya g' ada keluar , .

Topic 43: LinkAja saldo top-up issues
User Reviews:
1. s

In [ ]:
import re
from bs4 import BeautifulSoup
from IPython.display import HTML, display
import glob

def process_html_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    for details in soup.find_all('details'):
        if 'open' in details.attrs:
            del details['open']
    return str(soup)

def display_pages(html_contents):
    total_pages = len(html_contents)
    processed_contents = [process_html_content(content) for content in html_contents]

    navigation_html = f'''
    <div id="navigation" style="position: fixed; top: 10px; right: 10px; background-color: white; padding: 10px; border-radius: 5px; box-shadow: 0 0 10px rgba(0,0,0,0.1); display: flex; align-items: center;">
        <button onclick="changePage(-1)">Previous</button>
        <span id="currentPage" style="margin: 0 10px;">Page 1 of {total_pages}</span>
    <button onclick="changePage(1)">Next</button>
    </div>
    '''

    pages_html = ''.join([f'<div class="page" id="page{i+1}" style="display: none;">{content}</div>' for i, content in enumerate(processed_contents)])

    script = '''
    <script>
        var currentPage = 1;
        var totalPages = ''' + str(total_pages) + ''';

        function changePage(direction) {
            var newPage = currentPage + direction;
            if (newPage < 1) {
                newPage = totalPages;
            } else if (newPage > totalPages) {
                newPage = 1;
            }
            document.getElementById('page' + currentPage).style.display = 'none';
            document.getElementById('page' + newPage).style.display = 'block';
            currentPage = newPage;
            document.getElementById('currentPage').textContent = 'Page ' + currentPage + ' of ' + totalPages;
        }

        // Display the first page initially
        document.getElementById('page1').style.display = 'block';
    </script>
    '''

    combined_html = f'''
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; padding: 20px; }}
            h1, h2, h3 {{ color: #333; }}
            details {{ margin-bottom: 10px; }}
            summary {{ cursor: pointer; font-weight: bold; }}
        </style>
    </head>
    <body>
        {navigation_html}
        {pages_html}
        {script}
    </body>
    </html>
    '''

    display(HTML(combined_html))

# Baca semua file HTML dalam direktori saat ini tanpa output
html_contents = []
file_names = sorted(glob.glob('*.html'))

for file_name in file_names:
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            html_contents.append(file.read())
    except Exception:
        pass  # Abaikan file yang tidak bisa dibaca

# Gunakan fungsi untuk menampilkan halaman tanpa output tambahan
if html_contents:
    display_pages(html_contents)